In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy import signal
from scipy import fftpack
# import mne
from scipy.signal import iirnotch
from scipy import stats
from scipy.signal import butter, lfilter
from scipy.signal import freqs


In [6]:
channels = ['ch_'+str(i) for i in range(1,15)]

In [7]:
fs = 128 # Sampling frequency

In [8]:
#applying bandpass filter
def bandpass(data):
    # Define the filter parameters
    lowcut = 1  # Lower cutoff frequency
    highcut = 45  # Higher cutoff frequency
    order = 4  # Filter order

    # Calculate the Nyquist frequency
    nyq = 0.5 * fs

    # Calculate the filter cutoff frequencies in normalized units
    low = lowcut / nyq
    high = highcut / nyq

    # Design a Butterworth bandpass filter
    b, a = signal.butter(order, [low, high], btype='band')

    # Apply the filter to the EEG data using lfilter
    return signal.lfilter(b, a, data)

In [9]:
def notch(data):
    f0 = 50.0
    Q = 30.0

    # Compute the notch filter coefficients
    b, a = signal.iirnotch(f0, Q, fs)

    # Apply the notch filter to the signal
    return signal.filtfilt(b, a, data)

In [10]:
"""""
def butter_lowpass(cutOff, fs, order=5):
    nyq = 0.5 * fs
    normalCutoff = cutOff / nyq
    b, a = butter(order, normalCutoff, btype='low', analog = False)
    return b, a

def butter_lowpass_filter(data, cutOff, fs, order=4):
    b, a = butter_lowpass(cutOff, fs, order=order)
    y = lfilter(b, a, data)
    return y

def low_pass(data):
    cutOff = 40 #cutoff frequency in rad/s
    order = 20 #order of filter
    return  butter_lowpass_filter(data, cutOff, fs, order)
"""""

'""\ndef butter_lowpass(cutOff, fs, order=5):\n    nyq = 0.5 * fs\n    normalCutoff = cutOff / nyq\n    b, a = butter(order, normalCutoff, btype=\'low\', analog = False)\n    return b, a\n\ndef butter_lowpass_filter(data, cutOff, fs, order=4):\n    b, a = butter_lowpass(cutOff, fs, order=order)\n    y = lfilter(b, a, data)\n    return y\n\ndef low_pass(data):\n    cutOff = 40 #cutoff frequency in rad/s\n    order = 20 #order of filter\n    return  butter_lowpass_filter(data, cutOff, fs, order)\n'

In [12]:
import os
directory = 'datasets'
for filename in os.listdir(directory):
    eeg_data = pd.read_csv(f'{directory}/{filename}')
    label = eeg_data['label']
    preprocessed_data = pd.DataFrame()
    for i in channels:
        data = eeg_data[i]
        length = len(data)
        data = data - data.mean()
        data = notch(data)
        data = bandpass(data)
    #     data = low_pass(data)
        preprocessed_data[i] = data
#         fft(data)
    preprocessed_data['label'] = label
    preprocessed_data.to_csv(f'clean_data/{filename[:-4]}.csv',index = False)
    

     ---------------------------------------- 7.7/7.7 MB 97.1 kB/s eta 0:00:00
     -------------------------------------- 60.9/60.9 kB 129.5 kB/s eta 0:00:00
